<a href="https://colab.research.google.com/github/depradaboris/ProyectoPythonBigData/blob/main/laboratorio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os as os
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_de_acceso = '/content/drive/MyDrive/Dataset_de_ventas2'
#ruta_de_acceso = '/content/drive/MyDrive/UNI/ML/Dataset de ventas2'
archivos = os.listdir(ruta_de_acceso)

archivos

dataframes = {}

for archivo in archivos:
  if archivo.endswith('.csv'):
    mes = archivo.split('_')[3].split('.')[0]
    #print(mes)
    ruta_completa = os.path.join(ruta_de_acceso,archivo)
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy()

    #Transformo todos los valores no numericos en Nan en las columnas
    #Cantidad Pedida y Precio Unitario
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'],errors='coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'],errors='coerce')


    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    #Elimino todos los Nan y me quedo soolo con los valores numericos
    df_ventas = df_ventas.dropna()

    #Transformo todos los valores númericos al tipo de dato int
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)

    df_ventas['Ingreso generado'] = df_ventas['Cantidad Pedida']* df_ventas['Precio Unitario']

    dataframes[mes] = df_ventas



def new_colums_dia_mes_anio(dataFrameMes):
  dataFrameMes['Fecha de Pedido'] = pd.to_datetime(dataFrameMes['Fecha de Pedido'])

  # Convertir la columna a tipo datetime si aún no está en ese formato
  dataFrameMes['Fecha de Pedido']= pd.to_datetime(dataFrameMes['Fecha de Pedido'])

  # Obtener el día de cada timestamp y agregarlo como una nueva columna
  dataFrameMes['day'] = dataFrameMes['Fecha de Pedido'].dt.day

  dataFrameMes['month'] = dataFrameMes['Fecha de Pedido'].dt.month

  dataFrameMes['year'] = dataFrameMes['Fecha de Pedido'].dt.year

  return dataFrameMes

def get_city(dataFrameMes):
  #dt = dataFrameMes.groupby('Dirección de Envio')['Cantidad Pedida'].sum().reset_index()

  dataFrameMes['City'] = dataFrameMes['Dirección de Envio'].str.split(', ').str[1]# Obtengo la ciudad

  return dataFrameMes

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
   dataframe_mes = dataframes[mes]

In [ ]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  return datos

dataframe_mes = extraer_datos(dataframe_mes)

#1. Comportamiento de las ventas en los distintos meses:
● ¿Cómo variaron las ventas a lo largo de los diferentes meses?
● ¿Hubo algún mes que se destacó en términos de ventas?
● ¿Cuál es el ingreso total generado por mes?

In [ ]:
# Creo listas para almacenar los totales de ventas y los nombres de los meses
ventas_totales = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    ventas_totales_mes = dataframe_mes['Cantidad Pedida'].sum()

    ventas_totales.append(ventas_totales_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x = nombres_meses,
    y = ventas_totales,
    marker=dict(color=ventas_totales, colorscale = "RdYlGn", colorbar=dict(title="Cantidad", x=1.0, thickness=15)),  # Color y barra de colores
    text = ventas_totales,
    textposition = 'auto',  # Muestro el valor por encima de las barras

))


# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por Mes",
    xaxis=dict(title="MES"),
    yaxis=dict(title="Total de ventas"),
    yaxis_range = [0, max(ventas_totales)*1.1], #10% para que me muestre un poquito mas de lo max que llego el grafico
)

fig.show()

In [ ]:
# Creo listas para almacenar el ingreso total y los nombres de los meses
ingresos_totales = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    ingresos_totales_mes = dataframe_mes['Ingreso generado'].sum()

    ingresos_totales.append(ingresos_totales_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x = nombres_meses,
    y = ingresos_totales,
    marker=dict(color=ingresos_totales, colorscale = "RdBu", colorbar=dict(title="Ingresos Totales", x=1.0, thickness=15)),  # Color y barra de colores
    text = ingresos_totales,
    textposition = 'auto',  # Muestro el valor por encima de las barras

))


# Configuro los ejes y los títulos
fig.update_layout(
    title="Ingresos totales por Mes",
    xaxis=dict(title="MES"),
    yaxis=dict(title="Total de ingresos"),
    yaxis_range = [0, max(ingresos_totales)*1.1], #10% para que me muestre un poquito mas de lo max que llego el grafico
)

fig.show()

#2. Optimización de la publicidad y patrón de ventas por hora:
● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
probabilidad de compra?
● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?
● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
en los distintos meses?

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites
    Ventas_por_hora = dataframe_mes.groupby('Hora')['Cantidad Pedida'].count()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=Ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora seún",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()
horas_analizadas = [3, 11, 12, 19, 23]

for hora in horas_analizadas:
    total_productos_vendidos = []

    for mes in meses:
        dataframe_mes = dataframes[mes]
        dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites

        ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

        # Calcular la cantidad total de productos vendidos para la hora actual
        total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos,
        mode='lines',
        name=f'{hora} horas',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()


#3. Distribución de ventas por ubicación:
● ¿En qué ciudades se han registrado las mayores ventas?

● ¿Cómo se comparan las ventas en diferentes estados o regiones?

● ¿Existe variación en las ventas por estado a lo largo de los meses?

In [ ]:
#¿En qué ciudades se han registrado las mayores ventas?
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

dict_citys = {}

for mes in meses:
  dt=dataframes[mes]
  dataFrameWithCity = get_city(dt)
  print(dataFrameWithCity)
  dataFrameWithCity = dataFrameWithCity.groupby('City')['Cantidad Pedida'].sum().reset_index()
  dict_citys[mes]=dataFrameWithCity
  lista = dict_citys[mes]

cantidad_ciudades = len(dict_citys['Enero']['City'])

list_city = list()

for indice in range(0,cantidad_ciudades):
  city=dict_citys['Enero']['City'][indice]
  list_city.append(city)

list_sales_for_city = list()

for indice in range(cantidad_ciudades):
  suma=0
  for mes in meses:
    suma += dict_citys[mes]['Cantidad Pedida'][indice]
  list_sales_for_city.append(suma)

print(dataframes)

In [ ]:
# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=list_city,
    y=list_sales_for_city,
    marker=dict(color=list_sales_for_city, colorscale = 'RdYlGn', colorbar=dict(title="Cantidad de Ventas por Ciudad")),  # Color y barra de colores
    text=list_sales_for_city,
    textposition='auto',  # Muestro el valor encima de las barras

))

fig.update_layout(
    title="Cantidad de Ventas por Ciudad",
    xaxis=dict(title="Ciudades"),
    yaxis=dict(title="Ventas"),
    #height=800,  # Aumentar la altura
    width=1200,  # Aumentar el ancho
)

fig.show()

In [ ]:
#¿Cómo se comparan las ventas en diferentes estados o regiones?

dict_state = {}
dataframeMesState = {}
def get_state(dataFrameMes):
  dataFrameMes['State'] = dataFrameMes['Dirección de Envio'].str.split(', ').str[2]# Obtengo la ciudad
  dataFrameMes['State'] = dataFrameMes['State'].str.split(' ').str[0]

  return dataFrameMes

for mes in meses:
  dt=dataframes[mes]
  dataFrameWithState = get_state(dt)
  dataframeMesState[mes]= dataFrameWithState
  dataFrameWithState = dataFrameWithState.groupby('State')['Cantidad Pedida'].sum().reset_index()
  dict_state[mes]=dataFrameWithState


cantidad_states = len(dict_state['Enero']['State'])-1
list_state = list()

for indice in range(0,cantidad_states):
  state=dict_state['Enero']['State'][indice]
  list_state.append(state)

list_sales_for_state = list()

for indice in range(0,cantidad_states):
  suma=0
  for mes in meses:
    suma += dict_state[mes]['Cantidad Pedida'][indice]
  list_sales_for_state.append(suma)


#Empiezo a generar gráfico

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=list_state,
    y=list_sales_for_state,
    marker=dict(color=list_sales_for_state, colorscale = 'Viridis', colorbar=dict(title="Cantidad de Ventas por Ciudad")),  # Color y barra de colores
    text=list_sales_for_state,
    textposition='auto',  # Muestro el valor encima de las barras

))

fig.update_layout(
    title="Cantidad de Ventas por Estado",
    xaxis=dict(title="Estado"),
    yaxis=dict(title="States"),
    #height=800,  # Aumentar la altura
    width=1200,  # Aumentar el ancho
)

fig.show()

In [ ]:
import plotly.subplots as sp

def generate_more_graphips(list_ventas, list_days, mes):
    trace = go.Scatter(
        x=list_days,
        y=list_ventas,
        mode='lines+markers',
        line=dict(color='blue')
    )
    return trace

In [ ]:
#● ¿Existe variación en las ventas por estado a lo largo de los meses?
def state_for_mes(ventas,meses,estado):
  fig = go.Figure(data=go.Scatter(
  x=meses,
  y=ventas,
  mode='lines+markers',  # Para agregar puntos a la línea
  line=dict(color='blue')  # Cambiar el color de la línea
  ))

  # Personalizar el eje X y Y y dar un título al gráfico
  fig.update_xaxes(title_text='Months')
  fig.update_yaxes(title_text='Sales')
  fig.update_layout(title=name, xaxis_type='category')

  fig.show()

dataFrameMes = pd.DataFrame()
df = pd.DataFrame()
dict_meses_state = {}
for mes in meses:

  dataFrameMes = dict_state[mes]
  dict_meses_state[mes]=dataFrameMes.groupby('State')['Cantidad Pedida'].sum()

dt=dict_meses_state['Febrero'].reset_index()

list_name_state = list()
for i in range(0,len(dt['State'])):
  list_name_state.append(dt['State'][i])

print(list_name_state)

listCA=list()
listGA=list()

listME=list()
listMA=list()

listNY=list()
listTX=list()

listWA=list()
listOR=list()

dict_estado = {}
for mes in meses:
  dt=dict_meses_state[mes].reset_index()
  for i in range(0,len(dt['State'])):
    if dt['State'][i]=='CA':
      listCA.append(dt['Cantidad Pedida'][i])
      dict_estado['CA']=listCA

    #Función
    elif dt['State'][i]=='GA':
      listGA.append(dt['Cantidad Pedida'][i])
      dict_estado['GA']=listGA

    elif dt['State'][i]=='MA':
      listMA.append(dt['Cantidad Pedida'][i])
      dict_estado['MA']=listMA

    elif dt['State'][i]=='ME':
      listME.append(dt['Cantidad Pedida'][i])
      dict_estado['ME']=listME

    elif dt['State'][i]=='NY':
      listNY.append(dt['Cantidad Pedida'][i])
      dict_estado['NY']=listNY

    elif dt['State'][i]=='OR':
      listOR.append(dt['Cantidad Pedida'][i])
      dict_estado['OR']=listOR

    elif dt['State'][i]=='TX':
      listTX.append(dt['Cantidad Pedida'][i])
      dict_estado['TX']=listTX

    elif dt['State'][i]=='WA':
      listWA.append(dt['Cantidad Pedida'][i])
      dict_estado['WA']=listWA

print(dict_estado)
print(list_name_state)

for name in list_name_state:
  state_for_mes(dict_estado[name],meses,name)


['CA', 'GA', 'MA', 'ME', 'NY', 'OR', 'TX', 'WA']
{'CA': [4364, 5412, 6773, 8386, 7321, 6138, 6498, 5264, 5211, 9027, 7975, 11159], 'GA': [877, 1042, 1385, 1633, 1506, 1226, 1259, 1053, 1035, 1796, 1566, 2224], 'MA': [1171, 1401, 1884, 2190, 1985, 1616, 1771, 1409, 1455, 2445, 2172, 3029], 'ME': [138, 208, 214, 265, 241, 231, 219, 181, 161, 281, 259, 352], 'NY': [1497, 1807, 2215, 2741, 2556, 2001, 2169, 1804, 1711, 2953, 2650, 3828], 'OR': [553, 730, 901, 1134, 1067, 850, 808, 746, 668, 1276, 1063, 1507], 'TX': [1459, 1795, 2301, 2611, 2577, 1995, 2080, 1837, 1820, 3046, 2542, 3820], 'WA': [809, 1066, 1333, 1608, 1404, 1198, 1265, 1148, 1058, 1881, 1581, 2202]}
['CA', 'GA', 'MA', 'ME', 'NY', 'OR', 'TX', 'WA']


#4. Análisis del producto más vendido:
● ¿Cuál es el producto más vendido en general y en cada mes?
● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [ ]:
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

list_productos = list()
producto_mas_vendido = dict()
for mes in meses:
  data=dataframes[mes].groupby('Producto')['Cantidad Pedida'].sum()
  producto_mas_vendido[mes]=data

list_name_product = producto_mas_vendido['Febrero'].keys()
cantidad = len(list_name_product)

for indice in range(cantidad):
  suma=0
  for mes in meses:
    suma += producto_mas_vendido[mes][indice]
  list_productos.append(suma)



In [ ]:
# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=list_name_product,
    y=list_productos,
    marker=dict(color=list_productos, colorscale = 'RdYlGn', colorbar=dict(title="Cantidad de Productos Vendidos")),  # Color y barra de colores
    text=list_productos,
    textposition='auto',  # Muestro el valor encima de las barras

))

fig.update_layout(
    title="Cantidad de Productos Vendidos",
    xaxis=dict(title="Cantidad"),
    yaxis=dict(title="Producto"),
    #height=800,  # Aumentar la altura
    width=1200,  # Aumentar el ancho
)

fig.show()

y en cada mes?


In [ ]:

def generar_grafico_barar(list_name_product,lista_productos_mes,mes):
  # Creo el objeto figura con el gráfico de barras
  fig = go.Figure()

  fig.add_trace(go.Bar(
      x=list_name_product,
      y=lista_productos_mes,
      marker=dict(color=list_productos, colorscale = 'RdYlGn', colorbar=dict(title="Mes"+meses_l[mes])),  # Color y barra de colores
      #text=list_productos,
      #textposition='auto',  # Muestro el valor encima de las barras

  ))

  fig.show()


lista_mes = list()
for mes in meses:
  data=dataframes[mes].groupby('Producto')['Cantidad Pedida'].sum()
  lista_mes.append(data)


meses_l = {
    0: "Enero",
    1: "Febrero",
    2: "Marzo",
    3: "Abril",
    4: "Mayo",
    5: "Junio",
    6: "Julio",
    7: "Agosto",
    8: "Septiembre",
    9: "Octubre",
    10: "Noviembre",
    11: "Diciembre"
}

#for mes in range(12):
for i, mes in enumerate(range(12), start=1):
  lista_productos_mes = list()
  for productos in range(cantidad):
    lista_productos_mes.append(lista_mes[mes][productos])
  generar_grafico_barar(list_name_product,lista_productos_mes,mes)


# 5. Tendencia ventas:
● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
en los diferentes meses?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Diciembre']
def extraer_datoss(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Fin de Semana'] = datos['Fecha de Pedido'].dt.dayofweek.isin([5, 6])
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos
dataframe_mes = extraer_datoss(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()
fin_de_semana = dataframe_mes['Fin de Semana']

# Crear una columna en el DataFrame que identifique los días laborales y los fines de semana
dataframe_mes['Tipo de Día'] = dataframe_mes['Fin de Semana'].apply(lambda x: 'Fin de Semana' if x else 'Día Laboral')

# Calcular el total de ventas por día
ventas_totales_por_dia = dataframe_mes.groupby(['Dia', 'Tipo de Día'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras
fig = px.bar(ventas_totales_por_dia, x='Dia', y='Cantidad Pedida', color='Tipo de Día',
             labels={'Cantidad Pedida': 'Ventas'},
             category_orders={"Tipo de Día": ["Día Laboral", "Fin de Semana"]})

# Establecer los valores y etiquetas en el eje x
valores_x = list(range(1, len(ventas_totales_por_dia) + 1))
etiquetas_x = [str(i) for i in valores_x]

fig.update_xaxes(title_text='Día', tickvals=valores_x, ticktext=etiquetas_x)
fig.update_layout(title='Diciembre',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400,
                  )

fig.show()

#● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Diciembre']
def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Fin de Semana'] = datos['Fecha de Pedido'].dt.dayofweek.isin([5, 6])
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos
dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()
fin_de_semana = dataframe_mes['Fin de Semana']
import plotly.express as px

fig = px.bar(ventas_por_dia, x='Dia', y='Cantidad Pedida',
             labels={'Cantidad Pedida': 'Ventas'})

# Establecer los valores y etiquetas en el eje x
valores_x = list(range(1, len(ventas_por_dia) + 1))
etiquetas_x = [str(i) for i in valores_x]

fig.update_xaxes(title_text='Día', tickvals=valores_x, ticktext=etiquetas_x)
fig.update_layout(title='Diciembre',
                  yaxis_title='Ventas Total del Día',
                  width=650,
                  height=400,
                  )

# Agregar el total de ventas del día como etiquetas en las barras
fig.update_traces(text=ventas_por_dia['Cantidad Pedida'], textposition='inside')

fig.show()

#● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

In [ ]:
dataframe_mes['Fin de Semana'] = fin_de_semana

ventas_por_dia = ventas_por_dia.sort_values(by='Dia')

fig = px.bar(ventas_por_dia, x='Dia', y='Cantidad Pedida', barmode='group',
             labels={'Dia': 'Día', 'Cantidad Pedida': 'Ventas', 'Fin de Semana': 'Fin de Semana'})


fig.update_layout(title='Fin de Semana ',
                  xaxis_title='Día',
                  yaxis_title='Ventas Totales',
                  width=650,
                  height=400
                  )

fig.show()

### 6. Impacto de eventos especiales en las ventas:

● ¿Se ha observado algún aumento o disminución significativa en las ventas en días cercanos a eventos especiales, como días festivos?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Enero']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos
dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 1

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

# Crear una lista de días desde 0 hasta el último día
dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = px.bar(otros_dias, x='Dia', y='Cantidad Pedida',
             labels={'Cantidad Pedida': 'Ventas'})

# Agregar el día específico como una categoría adicional en el eje x
fig.add_bar(x=[dia_especifico], y=informacion_dia_especifico['Cantidad Pedida'],
            marker_color='red', name=f'Día {dia_especifico}')

# Actualizar el eje x para mostrar los días enumerados
fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el Día {dia_especifico} de Enero',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Julio']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos
dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 4

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

# Crear una lista de días desde 0 hasta el último día
dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = px.bar(otros_dias, x='Dia', y='Cantidad Pedida',
             labels={'Cantidad Pedida': 'Ventas'})

# Agregar el día específico como una categoría adicional en el eje x
fig.add_bar(x=[dia_especifico], y=informacion_dia_especifico['Cantidad Pedida'],
            marker_color='red', name=f'Día {dia_especifico}')

# Actualizar el eje x para mostrar los días enumerados
fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el día {dia_especifico} de Julio',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Noviembre']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos

dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 11

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = go.Figure(data=[go.Bar(x=otros_dias['Dia'], y=otros_dias['Cantidad Pedida'], name='Días')])

fig.add_trace(go.Bar(x=[dia_especifico], y=[informacion_dia_especifico['Cantidad Pedida'].iloc[0]],
                     marker_color='red', name=f'Día {dia_especifico}'))

# Agregar etiquetas de texto con el total de ventas en cada barra
fig.update_traces(text=otros_dias['Cantidad Pedida'], textposition='auto')

fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el Día {dia_especifico} de Noviembre',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Diciembre']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos

dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 25

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = go.Figure(data=[go.Bar(x=otros_dias['Dia'], y=otros_dias['Cantidad Pedida'], name='Días')])

fig.add_trace(go.Bar(x=[dia_especifico], y=[informacion_dia_especifico['Cantidad Pedida'].iloc[0]],
                     marker_color='red', name=f'Día {dia_especifico}'))

# Agregar etiquetas de texto con el total de ventas en cada barra
fig.update_traces(text=otros_dias['Cantidad Pedida'], textposition='auto')

fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el Día {dia_especifico} de Diciembre',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()


#¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
#las ventas y cómo se manifestó ese impacto

In [ ]:
import pandas as pd
import plotly.graph_objects as go

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Mayo']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos

dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 12

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = go.Figure(data=[go.Bar(x=otros_dias['Dia'], y=otros_dias['Cantidad Pedida'], name='Días')])

fig.add_trace(go.Bar(x=[dia_especifico], y=[informacion_dia_especifico['Cantidad Pedida'].iloc[0]],
                     marker_color='red', name=f'Día {dia_especifico}'))

# Agregar etiquetas de texto con el total de ventas en cada barra
fig.update_traces(text=otros_dias['Cantidad Pedida'], textposition='auto')

fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el Día {dia_especifico} de Mayo',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Junio']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos

dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 16

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = go.Figure(data=[go.Bar(x=otros_dias['Dia'], y=otros_dias['Cantidad Pedida'], name='Días')])

fig.add_trace(go.Bar(x=[dia_especifico], y=[informacion_dia_especifico['Cantidad Pedida'].iloc[0]],
                     marker_color='red', name=f'Día {dia_especifico}'))

# Agregar etiquetas de texto con el total de ventas en cada barra
fig.update_traces(text=otros_dias['Cantidad Pedida'], textposition='auto')

fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el Día {dia_especifico} de Junio',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes['Abril']

def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    datos['Mes'] = datos['Fecha de Pedido'].dt.month
    return datos

dataframe_mes = extraer_datos(dataframe_mes)
ventas_por_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum().reset_index()

dia_especifico = 21

informacion_dia_especifico = ventas_por_dia[ventas_por_dia['Dia'] == dia_especifico]

otros_dias = ventas_por_dia[ventas_por_dia['Dia'] != dia_especifico]

dias_enumerados = list(range(max(ventas_por_dia['Dia']) + 1))

fig = go.Figure(data=[go.Bar(x=otros_dias['Dia'], y=otros_dias['Cantidad Pedida'], name='Días')])

fig.add_trace(go.Bar(x=[dia_especifico], y=[informacion_dia_especifico['Cantidad Pedida'].iloc[0]],
                     marker_color='red', name=f'Día {dia_especifico}'))

# Agregar etiquetas de texto con el total de ventas en cada barra
fig.update_traces(text=otros_dias['Cantidad Pedida'], textposition='auto')

fig.update_xaxes(tickvals=dias_enumerados, ticktext=[str(d) for d in dias_enumerados])

fig.update_layout(title=f'Ventas para el Día {dia_especifico} de Abril',
                  xaxis_title='Día',
                  yaxis_title='Ventas Total del Día',
                  width=750,
                  height=400
                  )

fig.show()